In [2]:
import pyterrier as pt
import pandas as pd

In [4]:
indexer = pt.IterDictIndexer('./indices/msmarco-passage')
index = indexer.path
bm25 = pt.terrier.Retriever(index, wmodel="BM25", num_results=10) 

In [7]:
ds_topics = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')
ds_text = pt.get_dataset('irds:msmarco-passage')
topics = ds_topics.get_topics()

In [8]:
def get_text_for_docnos(docnos, dataset):
    """Efficiently get text for specific document IDs"""
    doc_texts = {}
    
    # Convert all docnos to the format we'll try
    docno_variants = {}
    for docno in docnos:
        str_docno = str(docno)
        docno_variants[str_docno] = docno
        # Also try integer version if it's numeric
        if str_docno.isdigit():
            docno_variants[int(str_docno)] = docno
    
    # Only iterate through corpus until we find all needed documents
    needed_ids = set(docno_variants.keys())
    found_count = 0
    
    corpus_iter = dataset.get_corpus_iter()
    for doc in corpus_iter:
        doc_id = doc['docno']
        
        # Check if this document is one we need
        if doc_id in needed_ids:
            original_docno = docno_variants[doc_id]
            doc_texts[original_docno] = doc['text']
            found_count += 1
            
            # Stop early if we found all documents we need
            if found_count >= len(docnos):
                break
    
    # Fill in missing documents
    for docno in docnos:
        if docno not in doc_texts:
            doc_texts[docno] = "Document text not available"
    
    return doc_texts

In [11]:
# Process each query with BM25 only, but also add text in the same format as MonoT5
all_results = []

for idx, row in topics.iterrows():
    query_id = row['qid']
    query_text = row['query']
    
    print(f"Processing query {idx+1}/{len(topics)}: {query_text}")
    
    # Get BM25 results - no reranking needed
    results = bm25.search(query_text)
    
    # Add text to the results
    docnos = results['docno'].tolist()
    doc_texts = get_text_for_docnos(docnos, ds_text)
    results['text'] = results['docno'].map(doc_texts)
    
    # Add query information to results
    results['qid'] = query_id
    results['query'] = query_text
    
    # Add docid (same as docno) to match MonoT5 format
    results['docid'] = results['docno']
    
    # Convert rank to 0-based if needed
    if 'rank' in results.columns and results['rank'].min() == 1:
        results['rank'] = results['rank'] - 1
    # Create rank column if missing
    elif 'rank' not in results.columns:
        results['rank'] = range(len(results))
    
    # Reorder columns to match MonoT5 CSV structure
    results = results[['qid', 'docid', 'docno', 'query', 'text', 'score', 'rank']]
    
    # Add to collection
    all_results.append(results)
    
    # Show progress for first query
    if idx == 0:
        print("\nSample results for first query:")
        print(results[['docno', 'score', 'text']].head(3))

# Combine all results
combined_results = pd.concat(all_results) if all_results else pd.DataFrame()

# Save results to CSV
combined_results.to_csv('bm25_results.csv', index=False)

print(f"\nRetrieved {len(combined_results)} documents across {len(topics)} queries")
print(f"Results saved to bm25_results.csv")

# Show summary of results
print("\nTop documents for first few queries:")
print(combined_results.groupby('qid').head(1)[['qid', 'docno', 'score']].head(5))

Processing query 1/43: do goldfish grow


msmarco-passage documents:   0%|          | 0/8841823 [00:00<?, ?it/s]

msmarco-passage documents:  93%|█████████▎| 8182166/8841823 [00:23<00:01, 341771.26it/s]



Sample results for first query:
     docno      score                                               text
0  8182161  31.156444  A: While goldfish can grow up to 18 inches in ...
1  6139386  31.032333  A: The conditions goldfish are kept in plus th...
2  3288600  30.953843  A goldfish will grow to the depth of the water...
Processing query 2/43: what is wifi vs bluetooth


msmarco-passage documents:  92%|█████████▏| 8160519/8841823 [00:25<00:02, 325800.25it/s]


...

Processing query 42/43: hydrogen is a liquid below what temperature


msmarco-passage documents:  89%|████████▉ | 7911557/8841823 [00:25<00:02, 312925.24it/s]


Processing query 43/43: difference between a mcdouble and a double cheeseburger


msmarco-passage documents:  95%|█████████▌| 8434626/8841823 [00:27<00:01, 309881.63it/s]


Retrieved 425 documents across 43 queries
Results saved to bm25_results.csv

Top documents for first few queries:
       qid    docno      score
0   156493  8182161  31.156444
0  1110199  3838645  36.361917
0  1063750  4788295  30.340022
0   130510  1494936  41.166240
0   489204  1479542  38.213874
